# LIBRARIES & FUNCTIONS

In [25]:
# LIBRARIES #

import numpy as np

from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.models import LinearColorMapper, BasicTicker, ColorBar, Plot, CustomJS, ColumnDataSource, Rect
from bokeh.layouts import row, gridplot, column
from bokeh.models.widgets import Slider, Button
from bokeh.events import ButtonClick

import SimpleITK as sitk 

from Gafchromic_multichannel import GafchromicFilms

import time


output_notebook()






# DISPLAY FUNCTIONS #

# Converts the tiff img in an rgba img #
# @params:
#   rgbaimg: array img to display
#   sizex: size of the img in x
#   sizey: size of the img in y
def convertToRGBA(array, sizex, sizey):

    # creates a new rgba img and copy the tiff values in it
    rgba = np.empty((sizey,sizex), dtype=np.uint32)
    view = rgba.view(dtype=np.uint8).reshape((sizey, sizex, 4))
    view[:,:,0] = array[:,:,0]/65535.0*255.0
    view[:,:,1] = array[:,:,1]/65535.0*255.0
    view[:,:,2] = array[:,:,2]/65535.0*255.0
    view[:,:,3] = 255

    return rgba




# Plots the gafchromic image and the rectangles listed in the rectList #
# @params:
#   rgbaimg: array img to display
#   sizex: size of the img in x
#   sizey: size of the img in y
#   rectlist: list of rectangles used for calibration
#   dimviewer: width of the image plot

def displayRectanglesOnImage(filename, rectlist, dimviewer) :
    img = sitk.ReadImage(filename)
    sizex = img.GetWidth()
    sizey = img.GetHeight()
    array = sitk.GetArrayFromImage(img)
    rgbaimg = convertToRGBA(array, sizex, sizey)
    
    p = figure(plot_width=int(dimviewer*0.95), plot_height=int(dimviewer*0.95*sizey/sizex), 
                    x_range=(0,sizex), y_range=(0,sizey), 
                    title="Gafchromic image", toolbar_location="above")
    
    p.image_rgba(image=[rgbaimg], x=[0], y=[0], dw=[sizex], dh=[sizey])

    for i in range(len(rectlist)):
        p.rect(x=int(rectlist[i][0]+(rectlist[i][2]-rectlist[i][0])/2), 
               y=int(rectlist[i][1]+(rectlist[i][3]-rectlist[i][1])/2), 
               width=rectlist[i][2]-rectlist[i][0],
               height=rectlist[i][3]-rectlist[i][1], 
               fill_color="#0096ff", 
               alpha=0.2)

    show(p)
    
    
    

# Plots the gafchromic image and the rectangles listed in the rectList #
# @params:
#   array: array img to display
#   dimviewer: width of the image plot
def basicImgDisplay(array, dimviewer=600) :

    sizex = array.shape[1]
    sizey = array.shape[0]

    rgbaimg = convertToRGBA(array, sizex, sizey)
    
    p = figure(plot_width=int(dimviewer*0.95), plot_height=int(dimviewer*0.95*sizey/sizex), 
                    x_range=(0,sizex), y_range=(0,sizey), 
                    title="Gafchromic image", toolbar_location="above")
    
    p.image_rgba(image=[rgbaimg], x=[0], y=[0], dw=[sizex], dh=[sizey])

    show(p)


    
    
    
# Plots the B&W image and the rectangles listed in the rectList #
# @params:
#   array: array img to display
#   dimviewer: width of the image plot
def basicBWImgDisplay(array, dimviewer=600, minvalue=0, maxv=0) :
    
    sizex = array.shape[1]
    sizey = array.shape[0]

    if (maxv==0): maxvalue = np.amax(img)
    else: maxvalue = maxv

    color_mapper = LinearColorMapper(palette="Viridis256", low=minvalue, high=maxvalue)
    
    color_bar = ColorBar(color_mapper=color_mapper, ticker=BasicTicker(),
                    label_standoff=12, border_line_color=None, location=(0,0),
                    title='')
    
    p = figure(plot_width=int(dimviewer*1.1), plot_height=int(dimviewer*sizey/sizex), 
                    x_range=(0,sizex), y_range=(0,sizey), 
                    title="Dose image", toolbar_location="above")
    
    p.image(image=[array], x=[0], y=[0], dw=[sizex], dh=[sizey], color_mapper=color_mapper)

    p.add_layout( color_bar, 'right')
    
    show(p)

    
    
    
    
# PLOTS IMAGE AND ONE X PROFILE #
# @params:
#   img: array img to display
#   sizex: size of the img in x
#   sizey: size of the img in y
#   linePos: position of the line profile
#   dSum: lines to use for the mean profile are the linePos +- dSum
#   imgPlotWidth: width of the image plot

def simpleImageDisplay(img, sizex, sizey, linePos, dSum=1, imgPlotWidth=500, LinePlotHeight = 300, minvalue=0, maxv=0):

    
    # Main image dose display:
    if (maxv==0): maxvalue = np.amax(img)
    else: maxvalue = maxv

    color_mapper = LinearColorMapper(palette="Viridis256", low=minvalue, high=maxvalue)
    
    color_bar = ColorBar(color_mapper=color_mapper, ticker=BasicTicker(),
                    label_standoff=12, border_line_color=None, location=(0,0),
                    title='')
    
    p1 = figure(plot_width=int(imgPlotWidth*1.1), plot_height=int(imgPlotWidth*sizey/sizex), 
                    x_range=(0,sizex), y_range=(0,sizey), 
                    title="Dose image", toolbar_location="above")
    
    p1.image(image=[img], x=[0], y=[0], dw=[sizex], dh=[sizey], color_mapper=color_mapper)
    p1.line(x=[0,sizex-1], y=[linePos, linePos], line_width=2, line_color="white")
    p1.rect(x=int(sizex/2), y=linePos, width=sizex, height=dSum*2+1, fill_color="white", alpha=0.2) 
    
    p1.add_layout( color_bar, 'right')
    
    # Line profile display:
    yLineDose = np.sum(img[linePos-dSum:linePos+dSum+1,:]/(2*dSum+1), axis=0)
    maxvalue = np.amax(yLineDose)
    minvalue = np.amin(yLineDose)
    
    p2 = figure(plot_width=int(imgPlotWidth*1.1), plot_height=int(LinePlotHeight), 
                    x_range=(0,sizex), y_range=(minvalue,maxvalue), 
                    title="Horizontal dose profile", toolbar_location="above")
    
    p2.line(x=range(sizex), y=yLineDose, line_color="#2690d4", line_width=3, line_alpha=1.0)
    
    # Grid:
    grid = gridplot([[p1],[p2]])
    show(grid)
    

    
    
# 2D visualisation of RGB values for a specific dose:
# @params:
#  arrA : x values
#  arrB : y values
#  rect : list of rectangles (where to find pixel values corresponding to a specific dose on the img)
#  doseValues : values of dose in the rectangles
def visu2D(arrA, arrB, rect, doseValues, title='2D scatter plot', titleX='X', titleY='Y'):
    
    p1 = figure(plot_width=700, plot_height=700, title=title, toolbar_location="above")

    listColors = ['cornflowerblue', 'darkorange', 'crimson', 'cadetblue', 'darkblue', 'darkgreen',
                 'darkgrey', 'darkmagenta', 'darkred', 'darkslategrey', 'firebrick', 'salmon', 'gold']

    for i in range(len(rectList)):
        rectValuesA = arrA[rect[i][1]:rect[i][3], rect[i][0]:rect[i][2]]
        rectValuesB = arrB[rect[i][1]:rect[i][3], rect[i][0]:rect[i][2]]
        p1.circle(rectValuesA.flatten(), rectValuesB.flatten(), size=3, color=listColors[i], 
                  alpha=0.5, legend_label=str(doseValues[i])+" Gy") 

    p1.xaxis.axis_label = titleX 
    p1.yaxis.axis_label = titleY 
    p1.legend.location = 'top_left' 

    show(p1)




Loading BokehJS ...

# IMG READING & PROCESSING

In [2]:
# INPUT PARAMETERS:
# <!> ne pas mettre d'accent dans les chemins et noms de fichiers


m_path = 'etalonnage 24h/'
m_nbOfFiles = 1
m_firstNb = 1
m_filesName = "etalonnage_24h_000"
m_fileExtension = ".tif"

m_dimViewer = 600


# Liste de dose (il est nécessaire de classer les dose dans un ordre croissant ou décroissant)
m_dose = [0, 25, 50, 100, 200, 300, 400, 500, 600, 800]


# Rectangles de l'image correspondant aux doses énoncées au dessus
rectList = []
rectList.append([1200,700,2000,900])
rectList.append([1200,1000,2000,1050])
rectList.append([1200,1300,2000,1500])
rectList.append([1200,1600,2000,1800])
rectList.append([1200,1900,2000,2100])
rectList.append([1200,2200,2000,2400])
rectList.append([1200,2500,2000,2700])
rectList.append([1200,2800,2000,3000])
rectList.append([1200,3110,2000,3310])
rectList.append([1200,3420,2000,3620])


#displayRectanglesOnImage(m_path+m_filesName+str(m_firstNb)+m_fileExtension, rectList, m_dimViewer)

print("dONE !")

dONE !


In [3]:
img = sitk.ReadImage(m_path+m_filesName+str(m_firstNb)+m_fileExtension)

sizex = img.GetWidth()
sizey = img.GetHeight()
imgOrigin = img.GetOrigin()
imgSpacing = img.GetSpacing()

size = (sitk.GetArrayFromImage(img).shape[0], 
        sitk.GetArrayFromImage(img).shape[1], 
        sitk.GetArrayFromImage(img).shape[2], 
        m_nbOfFiles)
imgs = np.zeros(size)

for i in range(m_nbOfFiles):
    img = sitk.ReadImage(m_path+m_filesName+str(m_firstNb+i)+m_fileExtension)
    imgs[:,:,:,i] = sitk.GetArrayFromImage(img)
    array = np.median(imgs, axis=3)

    
    
    

In [31]:
img = array[960:1240,1000:2200,:]

#basicImgDisplay(img, dimviewer=800)

minv = np.amin(img[:,:,2])
basicBWImgDisplay(img[:,:,2], dimviewer=800, minvalue = 30000, maxv = 40000)

In [ ]:
    
visu2D(array[:,:,1], array[:,:,2], rectList, m_dose,
      title='G over B values for different doses',
      titleX='G values',
      titleY='B values')

# Essais visualisation 3D

In [ ]:
from mpl_toolkits.mplot3d import Axes3D

import matplotlib.pyplot as plt
import numpy as np

#%matplotlib inline
%matplotlib widget





fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

for i in range(len(rectList)):
    rectValues = array[rectList[i][1]:rectList[i][3], rectList[i][0]:rectList[i][2], :]
    ax.scatter(rectValues[:,:,0].flatten(), rectValues[:,:,1].flatten(), 
               rectValues[:,:,2].flatten(), marker='o', color='darkblue')

ax.set_xlabel('X Label')
ax.set_ylabel('Y Label')
ax.set_zlabel('Z Label')

plt.show()